# Consideraciones para el entrenamiento

En el punto 5 del paper hablan de cómo han entrenado al transformer, así que vamos a verlo

## Datos

Para entrenar al transformer dicen que han usado el dataset [WMT 2014 English-German](https://aclanthology.org/W14-3302.pdf), pero mejor hacer un traductor de inglés a español, así que vamos a usar el dataset [opus100](https://huggingface.co/datasets/opus100) del inglés al español (`en-es`)

## Hardware

Han usado 8 Nvidia P100. Supongo que ninguno de nosotros tenemos acceso a 8 GPUs de ese tipo, por lo que haremos un código que sea ejecutable en [Colab](https://colab.research.google.com/), que al día de escribir este curso ofrece GPUs como la Nvidia T4 que tiene la misma VRAM que la P100, pero con una arquitectura más nueva. Por lo que en vez de usar 8 GPUs como las del paper, usaremos solo una pero un poco mejor que una de las que usaron ellos

## Steps

Dicen que entrenaron el modelo 100.000 pasos o 12 horas. Hay que diferenciar los steps de las epochs. Un step es cuando un solo batch size pasa por el modelo para entrenar y se realiza el backpropagation, mientras que una epoch es cuando todo el dataset ha pasado por el modelo.

Colab ofrece 12 horas de GPU, así que hacermos algo similar, entrenaremos unos 100.000 steps o cuando estemos un poco por debajo de las 12 horas pararemos el entrenamiento. Pero como nosotros solo vamos a poder usar una GPU seguramente no llegaremos a 100.000 steps

## Optimizador

Los investigadores dicen que usaron un optimizador Adam con $\beta_1 = 0.9$, $\beta_2 = 0.98$ y $\epsilon = 10^{-9}$ y que variaron el learning rate según la fórmula

$$lr = d_{model}^{-0.5} \cdot min \left(stepNumber^{-0.5}, stepNumber \cdot stepWarmUp^{-1.5} \right)$$

Esto corresponde a un aumento lineal de la tasa de aprendizaje para los primeros pasos de entrenamiento (pasos de calentamiento (warmup)), y a una disminución posterior proporcional a la raíz cuadrada inversa del número de pasos. Utilizaron 4000 pasos de calentamiento (warmup)

### Implementación del optimizador

Implementar el optimizador es sencillo, ya que como hemos visto ya que la función `torch.optim.Adam` hace todo, así que lo que tenemos que hacer es

``` python
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9)
```

### Implementación del scheduler del lr

Para hacer esto vamos a usar la función de Pytorch `torch.optim.lr_scheduler.LambdaLR`, que añade un scheduler al learning rate. Lo vamos a implementar de la siguiente manera

Primero creamos una función que calcula el valor que debe tener el learning rate en función de la época en la que está, es decir, creamos una función que resuelva la fórmula que indican en el paper

```python
def calculate_lr(step_num, dim_embeding_model=512, warmup_steps=4000):
    step_num += 1e-7 # Avoid division by zero
    return (dim_embeding_model**-0.5) * min(step_num**-0.5, step_num*(warmup_steps**-1.5))
```

Se puede ver que hemos añadido `step_num += 1e-7` porque si no, en la época 0 intenta hacer `step_num**-0.5` lo cual daría error porque no se puede elevar el número 0 a un número negativo

A continuación añadimos el scheduler del learning rate al optimizador que hemos creado antes

```python
lr_lambda = lambda step: calculate_lr(step, dim_embeding_model=dim_embedding)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
```

Por último, en el bucle de entrenamiento, después de hacer `optimizer.step()`, hay que además hacer `scheduler.step()` para modificar el valor del learning rate

## Regularización

Durante el entrenamiento utilizaron dropout con una probabilidad de 0.1 después de cada capa de atención y cada capa feed-forward. Además utilizaron label smoothing

El dropout ya lo hemos explicado, pero hacemos un recordatorio rápido. El dropout consiste en durante el entrenamiento desactivar la conexión entre neuronas aleatoriamente y con una probabilidad.

El label smoothing es un suavizado de las probabilidades de la secuencia de salida. Supongamos que le metemos al transformer la frase en español `¿Cuál es tu nombre?` y el trasnformer ha generado `What is your` de modo que la siguiente palabra tiene que ser `name`. Por lo que de todos los posibles tokens del inglés, el correspondiente a `name` debería tener la etiqueta de 1 y el resto de 0. Pero el label smoothing lo que hace es que name tenga una etiqueta de por ejemplo 0.95 y el resto 0.000001

Se define un valor de $1-\epsilon$, y se asigna la etiqueta del token que toca a $1-\epsilon$ y el resto de tokens tienen la etiqueta de $\epsilon / \left(numTokens-1 \right)$

### Implementación del dropout

Creamos una clase para el dropout

In [1]:
import torch

class Dropout(torch.nn.Module):
    def __init__(self, p=0.1):
        """
        Args:
            p (float): probability of an element to be zeroed. Default: 0.1
        """
        super().__init__()
        self.p = p

    def forward(self, x):
        """
        Args:
            x (torch.Tensor): input tensor

        Returns:
            torch.Tensor: a tensor with the same shape of `x`
        """
        if self.training:
            return torch.nn.functional.dropout(x, p=self.p)
        else:
            return x

### Implementación del label smoothing

Para poder implementar el label smoothing, simplemente se indica el valor de $\epsilon$ en la función de pérdida

```python
loss = nn.CrossEntropyLoss(ignore_index=pad_token, reduction='mean', label_smoothing=0.1)
```

Como vamos a usar la función de pérdida `nn.CrossEntropyLoss` ya no necesitamos usar la última capa de softmax del transformer, ya que esta función ya lo hace por nosotros, por lo que la clase `Linear_and_softmax` quedaría así

```python
class Linear_and_softmax(nn.Module):
    def __init__(self, dim_embedding, vocab_size):
        super().__init__()
        self.linear = CustomLinear(dim_embedding, vocab_size)
        # self.softmax = Softmax()
    
    def forward(self, x):
        x = self.linear(x)
        # x = self.softmax(x)
        return x
```

## Transformer

### Inicialización de los pesos

Para las clases `nn.Linear` vamos a inicializar los pesos según técnica de [Kaiming He](https://arxiv.org/pdf/1502.01852v1.pdf), que es una técnica de inicialización de los pesos para redes neuronales que tiene en cuenta la no linealidad de las funciones de activación, como las activaciones ReLU.
Por eso la vamos a usar en las capas `nn.Linear`, ya que a continuación de estas hay una activación `ReLU`

De modo que vamos a crear una nueva clase `CustomLinear` de la siguiente manera

```python
import torch
import torch.nn as nn
import torch.nn.init as init

class CustomLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(CustomLinear, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
        init.kaiming_uniform_(self.linear.weight, nonlinearity='relu')
        if self.linear.bias is not None:
            init.zeros_(self.linear.bias)
    
    def forward(self, x):
        return self.linear(x)
```

Para la clase `nn.Embedding` vamos a inicializar los pesos según la técnica de [Xavier Glorot](https://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf), que es un esquema de inicialización para redes neuronales. Los sesgos se inicializan en 0 y los pesos $W_{ij}$ en cada capa se inicializan como

$$ W_{ij} \sim U\left[-\frac{\sqrt{6}}{\sqrt{fan_{in} + fan_{out}}}, \frac{\sqrt{6}}{\sqrt{fan_{in} + fan_{out}}}\right] $$

Donde $U$ es una distribución uniforme y $fan_{in}$ es el tamaño de la capa anterior (número de columnas en $W$) y &fan_{out}$ es el tamaño de la capa actual

La principal ventaja de la inicialización de `Xavier` es que ayuda a combatir el problema del desvanecimiento o explosión de gradientes

De modo que vamos a crear una nueva clase `CustomEmbedding` de la siguiente manera

```python
import torch
import torch.nn as nn
import torch.nn.init as init

class CustomEmbedding(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super(CustomEmbedding, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        init.xavier_uniform_(self.embedding.weight)
    
    def forward(self, x):
        return self.embedding(x)
```

### Implementación del transformer

Volvemos a escribir cómo quedaría el transformer con el dropout, que es lo único que se añade al modelo

Primero escribimos todas las clases de bajo nivel

In [11]:
import torch
import torch.nn as nn
import torch.nn.init as init

class CustomLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(CustomLinear, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
        init.kaiming_uniform_(self.linear.weight, nonlinearity='relu')
        if self.linear.bias is not None:
            init.zeros_(self.linear.bias)
    
    def forward(self, x):
        return self.linear(x)

class CustomEmbedding(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super(CustomEmbedding, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        init.xavier_uniform_(self.embedding.weight)
    
    def forward(self, x):
        return self.embedding(x)

class Embedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        self.embedding = CustomEmbedding(vocab_size, embedding_dim)

    def forward(self, x):
        return self.embedding(x)

class PositionalEncoding(nn.Module):
    def __init__(self, max_sequence_len, embedding_model_dim):
        super().__init__()
        self.embedding_dim = embedding_model_dim
        positional_encoding = torch.zeros(max_sequence_len, self.embedding_dim)
        for pos in range(max_sequence_len):
            for i in range(0, self.embedding_dim, 2):
                positional_encoding[pos, i]     = torch.sin(torch.tensor(pos / (10000 ** ((2 * i) / self.embedding_dim))))
                positional_encoding[pos, i + 1] = torch.cos(torch.tensor(pos / (10000 ** ((2 * (i+1)) / self.embedding_dim))))
        positional_encoding = positional_encoding.unsqueeze(0)
        self.register_buffer('positional_encoding', positional_encoding)

    def forward(self, x):
        x = x * torch.sqrt(torch.tensor(self.embedding_dim))
        sequence_len = x.size(1)
        x = x + self.positional_encoding[:,:sequence_len]
        return x

class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim_embedding):
        super().__init__()
        self.dim_embedding = dim_embedding
    
    def forward(self, query, key, value, mask=None):
        key_trasposed = key.transpose(-1,-2)
        product = torch.matmul(query, key_trasposed)
        scale = product / torch.sqrt(torch.tensor(self.dim_embedding))
        if mask is not None:
            scale = scale.masked_fill(mask == 0, float('-inf'))
        attention_matrix = torch.softmax(scale, dim=-1)
        output = torch.matmul(attention_matrix, value)
        return output

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, dim_embedding):
        super().__init__()
        
        self.dim_embedding = dim_embedding
        self.dim_proyection = dim_embedding // heads
        self.heads = heads
        self.proyection_Q = CustomLinear(dim_embedding, dim_embedding)
        self.proyection_K = CustomLinear(dim_embedding, dim_embedding)
        self.proyection_V = CustomLinear(dim_embedding, dim_embedding)
        self.scaled_dot_product_attention = ScaledDotProductAttention(self.dim_proyection)
        self.attention = CustomLinear(dim_embedding, dim_embedding)
    
    def forward(self, Q, K, V, mask=None):
        batch_size = Q.size(0)
        proyection_Q = self.proyection_Q(Q).view(batch_size, self.heads, -1, self.dim_proyection)
        proyection_K = self.proyection_K(K).view(batch_size, self.heads, -1, self.dim_proyection)
        proyection_V = self.proyection_V(V).view(batch_size, self.heads, -1, self.dim_proyection)
        # proyection_Q = proyection_Q.transpose(1,2)
        # proyection_K = proyection_K.transpose(1,2)
        # proyection_V = proyection_V.transpose(1,2)
        scaled_dot_product_attention = self.scaled_dot_product_attention(proyection_Q, proyection_K, proyection_V, mask=mask)
        concat = scaled_dot_product_attention.transpose(1,2).contiguous().view(batch_size, -1, self.dim_embedding)
        output = self.attention(concat)
        return output

class AddAndNorm(nn.Module):
    def __init__(self, dim_embedding):
        super().__init__()
        self.normalization = nn.LayerNorm(dim_embedding)

    def forward(self, x, sublayer):
        return self.normalization(torch.add(x, sublayer))

class FeedForward(nn.Module):
    def __init__(self, dim_embedding, increment=4):
        super().__init__()
        self.feed_forward = nn.Sequential(
            CustomLinear(dim_embedding, dim_embedding*increment),
            nn.ReLU(),
            CustomLinear(dim_embedding*increment, dim_embedding)
        )
    
    def forward(self, x):
        x = self.feed_forward(x)
        return x

class Linear(nn.Module):
    def __init__(self, dim_embedding, vocab_size):
        super().__init__()
        self.linear = CustomLinear(dim_embedding, vocab_size)
        
    def forward(self, x):
        x = self.linear(x)
        return x

class Softmax(nn.Module):
    def __init__(self):
        super().__init__()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.softmax(x)
        return x

class Dropout(torch.nn.Module):
    def __init__(self, p=0.1):
        super().__init__()
        self.p = p

    def forward(self, x):
        if self.training:
            return torch.nn.functional.dropout(x, p=self.p)
        else:
            return x


Escribimos las clases de medio nivel

In [12]:
class EncoderLayer(nn.Module):
    def __init__(self, heads, dim_embedding, prob_dropout=0.1):
        super().__init__()
        self.multi_head_attention = MultiHeadAttention(heads, dim_embedding)
        self.dropout_1 = Dropout(prob_dropout)
        self.add_and_norm_1 = AddAndNorm(dim_embedding)
        self.feed_forward = FeedForward(dim_embedding)
        self.dropout_2 = Dropout(prob_dropout)
        self.add_and_norm_2 = AddAndNorm(dim_embedding)
    
    def forward(self, x):
        multi_head_attention = self.multi_head_attention(x, x, x)
        dropout1 = self.dropout_1(multi_head_attention)
        add_and_norm_1 = self.add_and_norm_1(x, dropout1)
        feed_forward = self.feed_forward(add_and_norm_1)
        dropout2 = self.dropout_2(feed_forward)
        add_and_norm_2 = self.add_and_norm_2(add_and_norm_1, dropout2)
        return add_and_norm_2

class Encoder(nn.Module):
    def __init__(self, heads, dim_embedding, Nx, prob_dropout=0.1):
        super().__init__()
        self.encoder_layers = nn.ModuleList([EncoderLayer(heads, dim_embedding, prob_dropout) for _ in range(Nx)])
    
    def forward(self, x):
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(x)
        return x

class DecoderLayer(nn.Module):
    def __init__(self, heads, dim_embedding, prob_dropout=0.1):
        super().__init__()
        self.masked_multi_head_attention = MultiHeadAttention(heads, dim_embedding)
        self.dropout_1 = Dropout(prob_dropout)
        self.add_and_norm_1 = AddAndNorm(dim_embedding)
        self.encoder_decoder_multi_head_attention = MultiHeadAttention(heads, dim_embedding)
        self.dropout_2 = Dropout(prob_dropout)
        self.add_and_norm_2 = AddAndNorm(dim_embedding)
        self.feed_forward = FeedForward(dim_embedding)
        self.dropout_3 = Dropout(prob_dropout)
        self.add_and_norm_3 = AddAndNorm(dim_embedding)
    
    def forward(self, x, encoder_output, mask=None):
        Q = x
        K = x
        V = x
        masked_multi_head_attention = self.masked_multi_head_attention(Q, K, V, mask=mask)
        dropout1 = self.dropout_1(masked_multi_head_attention)
        add_and_norm_1 = self.add_and_norm_1(dropout1, x)

        Q = add_and_norm_1
        K = encoder_output
        V = encoder_output
        encoder_decoder_multi_head_attention = self.encoder_decoder_multi_head_attention(Q, K, V)
        dropout2 = self.dropout_2(encoder_decoder_multi_head_attention)
        add_and_norm_2 = self.add_and_norm_2(dropout2, add_and_norm_1)

        feed_forward = self.feed_forward(add_and_norm_2)
        dropout3 = self.dropout_3(feed_forward)
        add_and_norm_3 = self.add_and_norm_3(dropout3, add_and_norm_2)

        return add_and_norm_3

class Decoder(nn.Module):
    def __init__(self, heads, dim_embedding, Nx, prob_dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([DecoderLayer(heads, dim_embedding, prob_dropout) for _ in range(Nx)])
    
    def forward(self, x, encoder_output, mask=None):
        for decoder_layer in self.layers:
            x = decoder_layer(x, encoder_output, mask)
        return x

class Linear_and_softmax(nn.Module):
    def __init__(self, dim_embedding, vocab_size):
        super().__init__()
        self.linear = CustomLinear(dim_embedding, vocab_size)
        # self.softmax = Softmax()
    
    def forward(self, x):
        x = self.linear(x)
        # x = self.softmax(x)
        return x


Y por último escribimos la clase transformer

In [13]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, src_max_seq_len, tgt_max_seq_len, dim_embedding, Nx, heads, prob_dropout=0.1):
        super().__init__()
        self.encoder = Encoder(heads, dim_embedding, Nx, prob_dropout)
        self.decoder = Decoder(heads, dim_embedding, Nx, prob_dropout)
        self.sourceEmbedding = Embedding(src_vocab_size, dim_embedding)
        self.targetEmbedding = Embedding(tgt_vocab_size, dim_embedding)
        self.sourcePositional_encoding = PositionalEncoding(src_max_seq_len, dim_embedding)
        self.targetPositional_encoding = PositionalEncoding(tgt_max_seq_len, dim_embedding)
        self.linear = Linear_and_softmax(dim_embedding, tgt_vocab_size)
    
    def encode(self, source):
        embedding = self.sourceEmbedding(source)
        positional_encoding = self.sourcePositional_encoding(embedding)
        encoder_output = self.encoder(positional_encoding)
        return encoder_output
    
    def decode(self, encoder_output, target, target_mask):
        embedding = self.targetEmbedding(target)
        positional_encoding = self.targetPositional_encoding(embedding)
        decoder_output = self.decoder(positional_encoding, encoder_output, target_mask)
        return decoder_output
    
    def projection(self, decoder_output):
        linear_output = self.linear(decoder_output)
        # softmax_output = self.softmax(linear_output)
        return linear_output
    
    def forward(self, source, target, target_mask):
        # Encode
        embedding_encoder = self.sourceEmbedding(source)
        positional_encoding_encoder = self.sourcePositional_encoding(embedding_encoder)
        encoder_output = self.encoder(positional_encoding_encoder)

        # Decode
        embedding_decoder = self.targetEmbedding(target)
        positional_encoding_decoder = self.targetPositional_encoding(embedding_decoder)
        decoder_output = self.decoder(positional_encoding_decoder, encoder_output, target_mask)

        # Projection
        proj_output = self.linear(decoder_output)
        return proj_output


Rescatamos las función que crea la máscara para el transformer

In [14]:
def create_mask(sequence_len):
    """
    Args:
        sequence_len: length of sequence
        
    Returns:
        mask matrix
    """
    mask = torch.tril(torch.ones((sequence_len, sequence_len)))
    return mask

Volvemos a definir la función que obtiene el embbeding más el positional encoding de BERT

In [15]:
import torch
from transformers import BertModel, BertTokenizer

def extract_embeddings(input_sentences, model_name='bert-base-uncased'):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    
    # tokenización de lote
    inputs = tokenizer(input_sentences, return_tensors='pt', padding=True, truncation=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
        
    token_embeddings = outputs[0]
    
    # Los embeddings posicionales están en la segunda capa de los embeddings de la arquitectura BERT
    positional_encodings = model.embeddings.position_embeddings.weight[:token_embeddings.shape[1], :].detach().unsqueeze(0).repeat(token_embeddings.shape[0], 1, 1)

    embeddings_with_positional_encoding = token_embeddings + positional_encodings

    # convierte las IDs de los tokens a tokens
    tokens = [tokenizer.convert_ids_to_tokens(input_id) for input_id in inputs['input_ids']]

    return tokens, inputs['input_ids'], token_embeddings, positional_encodings, embeddings_with_positional_encoding

Creamos una sentencia para el encoder y otra para el decoder

In [16]:
sentence_encoder = "I gave the dog a bone because it was hungry"
sentence_decoder = "Le di un hueso al perro porque tenía hambre"

tokens_encoder, input_ids_encoder, token_embeddings_encoder, positional_encodings_encoder, embeddings_with_positional_encoding_encoder = extract_embeddings(sentence_encoder)
tokens_decoder, input_ids_decoder, token_embeddings_decoder, positional_encodings_decoder, embeddings_with_positional_encoding_decoder = extract_embeddings(sentence_decoder)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.pred

Creamos un objeto `transformer` y obtenemos su salida

In [20]:
src_vocab_size = 30522
tgt_vocab_size = 30522
dim_embedding = max(token_embeddings_encoder.shape[-1], token_embeddings_decoder.shape[-1])
src_max_sequence_len = token_embeddings_encoder.shape[1]
tgt_max_sequence_len = token_embeddings_decoder.shape[1]
heads = 8
Nx = 6
print(f"src vocab_size: {src_vocab_size}, dim_embedding: {dim_embedding}, src max_sequence_len: {src_max_sequence_len}, heads: {heads}, Nx: {Nx}")
print(f"tgt vocab_size: {tgt_vocab_size}, dim_embedding: {dim_embedding}, tgt max_sequence_len: {tgt_max_sequence_len}, heads: {heads}, Nx: {Nx}")
transformer = Transformer(src_vocab_size, tgt_vocab_size, src_max_sequence_len, tgt_max_sequence_len, dim_embedding, Nx, heads)

src vocab_size: 30522, dim_embedding: 768, src max_sequence_len: 12, heads: 8, Nx: 6
tgt vocab_size: 30522, dim_embedding: 768, tgt max_sequence_len: 16, heads: 8, Nx: 6


In [21]:
mask = create_mask(tgt_max_sequence_len)
mask.shape

torch.Size([16, 16])

In [22]:
transformer_output = transformer(input_ids_encoder, input_ids_decoder, target_mask=mask)
transformer_output.shape

torch.Size([1, 16, 30522])

Al igual que en el notebook anterior obtenemos 1x16x30522, es decir 1 batch, 16 tokens y 30522 posibles tokens del vocabulario